In [13]:
import os
import torch
import chromadb
from transformers import AutoTokenizer, AutoModel
import json

In [37]:
class RuModernBERTEmbedding:
    """Функция эмбеддинга на основе deepvk/RuModernBERT-small для использования в ChromaDB"""

    def __init__(self, model_name="deepvk/RuModernBERT-small"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def get_embeddings(self, texts):
        """Генерирует эмбеддинги для списка текстов"""
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.cpu().numpy().tolist()  # Преобразуем в список списков

# Создаем клиент ChromaDB с сохранением данных
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Создаем коллекцию (если её нет, создаст новую)
collection_name = "text_embeddings"
collection = chroma_client.get_or_create_collection(name=collection_name)

# Создаем объект эмбеддинга
embedding_function = RuModernBERTEmbedding()

# Путь к папке с текстовыми файлами
folder_path = 'Annotations'

# Получение списка всех файлов в папке
file_names = os.listdir(folder_path)

# Читаем содержимое файлов
documents = []
ids = []

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        documents.append(text)
        ids.append(file_name)  # Используем имя файла как ID

# Генерируем эмбеддинги
embeddings = embedding_function.get_embeddings(documents)

# Добавляем данные в ChromaDB
collection.add(
    ids=ids,
    documents=documents,
    embeddings=embeddings
)

print(f"Добавлено {len(documents)} документов в коллекцию '{collection_name}'.")



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Add of existing embedding ID: text1.txt
Add of existing embedding ID: text2.txt
Insert of existing embedding ID: text1.txt
Insert of existing embedding ID: text2.txt


Добавлено 2 документов в коллекцию 'text_embeddings'.


In [46]:
# Поиск по векторной базе
query = "Жизнь российского писателя?"
search_results = collection.query(
    query_embeddings=embedding_function.get_embeddings([query]),  # Генерируем эмбеддинг запроса
    n_results=2
)

# Извлекаем нужные данные
ids = search_results.get("ids", [])
documents = search_results.get("documents", [])
distances = search_results.get("distances", [])

# Выводим результаты
print("IDs:", ids)
print("Documents:", documents)
print("Distances:", distances)


IDs: [['text1.txt', 'text2.txt']]
Documents: [['Аннотация к судебному решению по делу номер А41-93574 2024. Истец индивидуальный предприниматель Антонова ЮА подала иск к ООО РВБ о взыскании неосновательного обогащения в размере 300000 рублей, процентов за пользование чужими денежными средствами в размере 13267 рублей 79 копеек, расходов на оплату услуг представителя в размере 55000 рублей и государственной пошлины в размере 20663 рублей. Основанием для иска стало удержание маркетплейсом Вайлдберриз штрафов за нарушение правил использования площадки, а именно за использование механик искусственного завышения рейтинга через самовыкупы товаров. Истец утверждает, что штрафы в размере 300000 рублей были начислены необоснованно и самовыкупы не совершались, в связи с чем требует возврата удержанных средств как неосновательного обогащения. Ответчик ООО РВБ и третье лицо Вайлдберриз утверждают, что штрафы были начислены правомерно в соответствии с условиями оферты, запрещающей манипулирование р